In [21]:
import numpy as np
import pandas as pd
import re
from datetime import datetime
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

In [22]:
# key revenue metrix
# https://portraits.core.tf/app/kibana#/visualize/create?type=histogram&indexPattern=portraits&_g=(refreshInterval:(display:Off,pause:!f,value:0),time:(from:now-15m,mode:quick,to:now))&_a=(filters:!(),linked:!f,query:(query_string:(analyze_wildcard:!t,query:'*')),uiState:(spy:(mode:(fill:!f,name:table))),vis:(aggs:!((id:'1',params:(),schema:metric,type:count),(id:'4',params:(filters:!(('$$hashKey':'object:2155',input:(query:(query_string:(analyze_wildcard:!t,query:'*'))),label:''),('$$hashKey':'object:2165',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_cents')))))),schema:group,type:filters),(id:'2',params:(filters:!(('$$hashKey':'object:392',input:(query:(query_string:(analyze_wildcard:!t,query:'tags:Multirun_web_3_1_express_disabled_run1__TG1'))),label:''),('$$hashKey':'object:401',input:(query:(query_string:(analyze_wildcard:!t,query:'tags:Multirun_web_3_1_express_disabled_run1__CG1')))),('$$hashKey':'object:406',input:(query:(query_string:(analyze_wildcard:!t,query:'tags:Multirun_web_3_1_express_disabled_run1__CG2')))))),schema:segment,type:filters),(id:'3',params:(field:revenue_in_cents),schema:metric,type:sum)),listeners:(),params:(addLegend:!t,addTimeMarker:!f,addTooltip:!t,defaultYExtents:!f,mode:stacked,scale:linear,setYExtents:!f,shareYAxis:!t,times:!(),yAxis:()),title:'New%20Visualization',type:histogram))
# R21 / R14
# https://portraits.core.tf/app/kibana#/visualize/create?type=histogram&indexPattern=portraits&_g=(refreshInterval:(display:Off,pause:!f,value:0),time:(from:now-15m,mode:quick,to:now))&_a=(filters:!(),linked:!f,query:(query_string:(analyze_wildcard:!t,query:'*')),uiState:(spy:(mode:(fill:!f,name:table))),vis:(aggs:!((id:'1',params:(),schema:metric,type:count),(id:'4',params:(filters:!(('$$hashKey':'object:2155',input:(query:(query_string:(analyze_wildcard:!t,query:'*'))),label:''),('$$hashKey':'object:2165',input:(query:(query_string:(analyze_wildcard:!t,query:'retention_days:14')))),('$$hashKey':'object:2330',input:(query:(query_string:(analyze_wildcard:!t,query:'retention_days:21')))))),schema:group,type:filters),(id:'2',params:(filters:!(('$$hashKey':'object:392',input:(query:(query_string:(analyze_wildcard:!t,query:'tags:Multirun_web_3_1_express_disabled_run1__TG1'))),label:''),('$$hashKey':'object:401',input:(query:(query_string:(analyze_wildcard:!t,query:'tags:Multirun_web_3_1_express_disabled_run1__CG1')))),('$$hashKey':'object:406',input:(query:(query_string:(analyze_wildcard:!t,query:'tags:Multirun_web_3_1_express_disabled_run1__CG2')))))),schema:segment,type:filters)),listeners:(),params:(addLegend:!t,addTimeMarker:!f,addTooltip:!t,defaultYExtents:!f,mode:stacked,scale:linear,setYExtents:!f,shareYAxis:!t,times:!(),yAxis:()),title:'New%20Visualization',type:histogram))


In [23]:
### Load payments
df_payments_TG1 = pd.read_csv(r'./Multirun_web_3_1_express_disabled_run1__TG1.csv')
df_payments_CG1 = pd.read_csv(r'./Multirun_web_3_1_express_disabled_run1__CG1.csv')
df_payments_CG2 = pd.read_csv(r'./Multirun_web_3_1_express_disabled_run1__CG2.csv')
df_payments_TG1['group'] = 'TG1'
df_payments_CG1['group'] = 'CG1'
df_payments_CG2['group'] = 'CG2'
df_payments = pd.concat([df_payments_TG1, df_payments_CG1, df_payments_CG2])

# df = df[df['registration date'] < border_install_date]
df_payments['payment date'] = df_payments['payment date'].map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
df_payments['registration date'] = df_payments['registration date'].map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))  
df_payments['payment day'] = df_payments['payment date'].map(lambda x: x.strftime('%Y-%m-%d'))
df_payments['registration day'] = df_payments['registration date'].map(lambda x: x.strftime('%Y-%m-%d'))
df_payments['registration month'] = df_payments['registration date'].map(lambda x: x.strftime('%Y-%m'))
df_payments['time from registration'] = df_payments['payment date'] - df_payments['registration date']
df_payments['days from registration'] = df_payments['time from registration'].map(lambda x: x.days)

In [24]:
df_payments.head()

,ref ID,partner ID,ref date,user ID,registration date,payment ID,payment date,payment amount,revenue in cents,payment service ID,...,place,country,last app,Product Id,group,payment day,registration day,registration month,time from registration,days from registration
0,597.0,12.0,2018-03-21 21:58:51,117328667,2018-03-21 21:58:51,1,2018-03-21 22:08:50,100,97,19,...,coins_popup_navigation,US,standalonetf,coin.one,TG1,2018-03-21,2018-03-21,2018-03,0 days 00:09:59,0
1,NaN,NaN,NaN,117191687,2018-03-17 17:01:17,5,2018-03-18 06:07:28,17,8,57,...,superPopupFromBlockExpressMessages,KZ,topface,premium.1day_PUincrease,TG1,2018-03-18,2018-03-17,2018-03,0 days 13:06:11,0
2,597.0,12.0,2018-03-17 20:53:16,117195121,2018-03-17 20:53:16,8,2018-03-20 10:24:12,203,142,147,...,FromMenu,UG,topfacemobappios,com.topface.topface.premium.week.1,TG1,2018-03-20,2018-03-17,2018-03,2 days 13:30:56,2
3,597.0,12.0,2018-03-12 02:28:47,117095065,2018-03-12 02:28:46,5,2018-03-12 12:04:39,308,172,19,...,sympathy_access,RU,standalonetf,NaN,TG1,2018-03-12,2018-03-12,2018-03,0 days 09:35:53,0
4,NaN,NaN,NaN,117092298,2018-03-11 21:03:53,23,2018-03-23 09:05:58,100,97,19,...,coins_popup_navigation,BY,standalonetf,coin.one,TG1,2018-03-23,2018-03-11,2018-03,11 days 12:02:05,11


In [25]:
# 19 - Покупка монеток - PaymentServiceBuyCoins
# 57 - Покупка Premium-статуса - PaymentServicePremium
# 67 - Плата за покупку лайков - PaymentServiceLikes
# 68 - Покупка премиума за бабло - PaymentServiceBuyPremium
# 77 - Плата за покупку бомбы сообщений - Topface.Product.Payment.PaymentServiceBombMessage
# 127 - Автоматическая покупка монет - Topface.Subscription.PaymentWall.AutoRechargeCoinsPaymentService
# 146 - Покупка подписки на монеты - PaymentServiceCoinSubscription
# 147 - Покупка подписки на премиум - PaymentServicePremiumSubscription

In [26]:
df = df_payments[df_payments['days from registration'] >= 14]

In [27]:
df.groupby(['group'])['revenue in cents'].sum()

group
CG1    7470
CG2    5191
TG1    6649
Name: revenue in cents, dtype: int64

In [28]:
df[df['payment service ID'].isin([57, 68, 147])].groupby(['group'])['revenue in cents'].sum()

group
CG1    2875
CG2    4263
TG1    4649
Name: revenue in cents, dtype: int64

In [29]:
df[df['payment service ID'].isin([57, 147])].groupby(['group'])['revenue in cents'].sum()

group
CG1     534
CG2     668
TG1    3766
Name: revenue in cents, dtype: int64

In [30]:
df[df['Product Id'].fillna('none').map(lambda x: bool(re.search('prem',x)))].groupby(['group'])['revenue in cents'].sum()

group
CG1    2875
CG2    3814
TG1    4003
Name: revenue in cents, dtype: int64

In [31]:
df[df['place'].isin(['super_express_message','superPopupFromBlockExpressMessages'])].groupby(['group'])['revenue in cents'].sum()

group
CG1     296
CG2    2793
Name: revenue in cents, dtype: int64

In [34]:
# 68 - Покупка премиума за бабло - PaymentServiceBuyPremium
# df[(df['payment service ID'] == 68) & (df['group'] == 'TG1')][['Product Id','payment amount', 'payment date', 'payment provider','payment service ID', 'place', 'platform','registration date', 'revenue in cents', 'user ID', 'payment day','registration day']]